## Clustering of ligand cavity shape

In this tutorial, we will perform clustering of ligand cavity. This tutorial can be used as a giude **to perform protein channel shape clustering** to analyse different shapes of channel during the simulations. The protein considered in this tutorial has ligand cavity burried deep inside the protein and cavity is almost linear.

### Final result

<img src="https://raw.githubusercontent.com/rjdkmr/gmx_clusterByFeatures/master/docs/images/cavity_clustering_final_result.png" width="1200"/>

### Instructions
* **Tutorial files**: The tutorial files can be downloaded from [here](https://figshare.com/ndownloader/files/55894172).
* **Extract the files**: `tar -zxvf protein-pca.tar.gz`
* **Go to directory**: `cd protein-pca`
* **Copy the Jupyter Notebook**: This notebook is available in the GitHub repo. [Download and copy it from the github](https://github.com/rjdkmr/gmx_clusterByFeatures/tree/master/docs/tutorials).


### Required Tools
* [HOLE2 - A tool to calculate channel/cavity radius](https://www.holeprogram.org/)
* GROMACS
* gmx_clusterByFeatures

### Steps
1. Preparation of reference structure
2. Generate a new trajectory aligned on reference structure
3. Plotting radius and residues distribution
4. Calculation of cavity/channel radius
5. Create feature-file
6. Clustering using radius as features
7. Analysis

### 1. Preparation of reference structure

A reference structure need to be prepared where cavity/channel axis align with one of the principal axis. For protein channel, it might be already done as channel-axis is mostly aligned along z-axis. In this case, I have already aligned the ligand-cavity axis along the Y-axis and it is provided as `inputs/ref_cavity.pdb`.

<img src="https://raw.githubusercontent.com/rjdkmr/gmx_clusterByFeatures/master/docs/images/cavity_axis_aligned.png" width="700"/>

### 2. Generate a new trajectory superimposed on reference structure

`gmx_clusterByFeatures hole` has a functionality to superimpose the conformation for consistent calculation of cavity/channel radius along the given vector However, during clustering, the output central structures and cluster trajectory will be not superimposed and therefore, it is good to superimpose whole trajectory on reference structure so that cluster's central structures orientations will be same as of reference structure.

We will use `gmx trjconv` to create a new superimposed trajectory with reference structre as follows.


In [3]:
%%bash

rm traj_superimposed.xtc

echo 11 0 | gmx trjconv -s inputs/ref_cavity.pdb -f inputs/onlyProtein.xtc -n inputs/onlyProtein.ndx \
                        -fit rot+trans -o traj_superimposed.xtc

                     :-) GROMACS - gmx trjconv, 2025.2 (-:

Executable:   /opt/gromacs-2025/bin/gmx
Data prefix:  /opt/gromacs-2025
Working dir:  /home/raj/workspace/gmx_clusterByFeatrues/tutorials/protein-pca
Command line:
  gmx trjconv -s inputs/ref_cavity.pdb -f inputs/onlyProtein.xtc -n inputs/onlyProtein.ndx -fit rot+trans -o traj_superimposed.xtc

Will write xtc: Compressed trajectory (portable xdr format): xtc

         that are broken across periodic boundaries, they
         cannot be made whole (or treated as whole) without
         you providing a run input file.

Group     0 (         System) has  8327 elements
Group     1 (        Protein) has  8327 elements
Group     2 (      Protein-H) has  4217 elements
Group     3 (        C-alpha) has   542 elements
Group     4 (       Backbone) has  1626 elements
Group     5 (      MainChain) has  2169 elements
Group     6 (   MainChain+Cb) has  2660 elements
Group     7 (    MainChain+H) has  2668 elements
Group     8 (      SideCha

Note that major changes are planned in future for trjconv, to improve usability and utility.

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

Select group for least squares fit
Selected 11: 'r_5-535_&_MainChain_&_!r_72-85_&_!r_257-268_&_!r_338-352_&_!r_435-444'
Select group for output
Selected 0: 'System'


### 3. Calculation of cavity/channel radius

We will use [HOLE2](https://www.holeprogram.org/) tool to calculate ligand cavity. It should be installed and `hole` should be in PATH. HOLE accept only one pdb file as input. `gmx_clusterByFeatures hole` use external `hole` and calculate radius from each frame automating the entire calculation for the trajectory.

Following options are used here:
* `-catmid 1903` - seed atom from where radius calculation will start in both the direction along the axis
* `-rad bondi` - radius of atoms
* `-cvect 0 1 0` - vector along which radius will be calculated. Here, it is Y-axis.
* `-endrad 12` - stop radius calculation when this threshold radius is reached. It means, now it is in solvent phase.
* `-sample 0.5` - slab size along the cavity/channel axis. Radius is calculated in each slab
* `-mcstep 2000` - Monte-Carlo step size. See details in HOLE manual

This command will dump radius data as a function of time in `radius.dat` file

In [1]:
%%bash

rm radius.dat

echo 11 1 | gmx_clusterByFeatures hole -f traj_superimposed.xtc -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx \
                                       -rad bondi -catmid 1903 -cvect 0 1 0 -endrad 12 -sample 0.5 -mcstep 2000

 :-) GROMACS - gmx_clusterByFeatures hole, 2025.0-dev-20250210-6949615-local (-:

Executable:   gmx_clusterByFeatures hole
Data prefix:  /project/external/gmx_installed
Working dir:  /home/raj/workspace/gmx_clusterByFeatrues/tutorials/protein-pca
Command line:
  'gmx_clusterByFeatures hole' -f traj_superimposed.xtc -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx -rad bondi -catmid 1903 -cvect 0 1 0 -endrad 12 -sample 0.5 -mcstep 2000


        :-)  gmx_clusterByFeatures hole (-:
        
        Author: Rajendra Kumar
        
        Copyright (C) 2018-2025  Rajendra Kumar
        
        
        gmx_clusterByFeatures is a free software: you can redistribute it and/or modify
        it under the terms of the GNU General Public License as published by
        the Free Software Foundation, either version 3 of the License, or
        (at your option) any later version.
        
        gmx_clusterByFeatures is distributed in the hope that it will be useful,
        but WITHOUT ANY W


Choose a group for the least squares fit
Selected 11: 'r_5-535_&_MainChain_&_!r_72-85_&_!r_257-268_&_!r_338-352_&_!r_435-444'

Choose a group for hole calculation...
Selected 1: 'Protein'


Selected "H" atom of "GLY-122" as seed for channel/cavity

Thanks for using gmx_hole!!!

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++


### 4. Plotting radius and residues distribution

To plot radius value distributions and residues outlining cavity/channel positions distribution over the simulations, 
`gmx_clusterByFeatures holeplot` sub-command can be used. It reads radius file generated in the above command, extract
radius values for the given segment of the cavity/channel, calculate average/distributions and plot it as a function
of cavity/channel axis.

**Options**
* `-resplot` - enable plotting of residues positions distributions
* `-vplot` - distributions are plotted as violin-plot
* `-xmin` - minimum value of cavity/channel axis position to consider
* `-xmax` - maximum value of cavity/channel axis position to consider
* `-endrad` - ignore if radius is greater than this value (mostly at the opening end)
* `-gap` - slab size along the cavity/channel
* `-ax` - X, Y or Z axis along which cavity/channel axis is aligned
* `-rlsize` - font-size of residues in residue-distribution plots.

In [1]:
%%bash

gmx_clusterByFeatures holeplot -i radius.dat -resplot -vplot -o average_radius_distribution.png \
                               -xmin 0 -xmax 17 -endrad 6 -gap 0.5 -ax Y -rlsize 7


Reading file : radius.dat
Reading frame 0 at time        0.000
Maximum axis range is: (0.000 to 17.000)
Reading frame 20000 at time  2000000.000
Finishid reading.... Total number of frame read =  22501
After removing axis points with any missing data:
------------------------------------------------------------
#        Axis        Mean radius     Std. deviation
------------------------------------------------------------
       0.000            0.864            0.286
       0.500            0.867            0.336
       1.000            0.889            0.394
       1.500            0.904            0.442
       2.000            0.920            0.460
       2.500            0.958            0.449
       3.000            0.993            0.455
       3.500            1.043            0.466
       4.000            1.144            0.448
       4.500            1.288            0.414
       5.000            1.449            0.386
       5.500            1.614            0.387
       6.

<img src="average_radius_distribution.png" width="900"/> 

**Observations**

As can be seen aboive, radius of cavity flucuated by at-least 2Å and much more along the cavity.
Therefore, it does not provide a true picture of different shapes sampled during the simulations.
Clustering could unravel all those shapes sampled during the simulations.

### 5. Create feature file

In the next step, we will create a file containing cavity radius as features for clustering. `gmx_clusterByFeatures holefeatures` can be used to create this file using the input `radius.dat` file, which is obtained in the previous step.

**Options**
* `-xmin` Minimum axis position value above which radius will be considered. Here, we take it as 5.5 because GLY122 is present at the bottom most region of the cavity and residue-distribution shows that GLY122 postions start from 5.5Å along the axis.
* `-xmax` Maximum axis position value below which radius will be considered
* `-endrad` Maximum radius to be considered for opening end of the cavity
* `-gap` Radius calculation slab size
* `-ax` Cavity/Channel axis

In [2]:
%%bash

gmx_clusterByFeatures holefeatures -i radius.dat -o cavity_features.xvg \
                                   -xmin 5.5 -xmax 17 -endrad 6 -gap 0.5 -ax Y


Reading file : radius.dat
Reading frame 0 at time        0.000
Maximum axis range is: (5.500 to 17.000)
Reading frame 20000 at time  2000000.000
Finishid reading.... Total number of frame read =  22501
After removing axis points with any missing data:
------------------------------------------------------------
#        Axis        Mean radius     Std. deviation
------------------------------------------------------------
       5.500            1.629            0.384
       6.000            1.750            0.451
       6.500            1.907            0.534
       7.000            2.081            0.594
       7.500            2.146            0.631
       8.000            2.142            0.637
       8.500            2.127            0.610
       9.000            2.111            0.581
       9.500            2.085            0.569
      10.000            2.036            0.571
      10.500            1.984            0.567
      11.000            1.948            0.559
      11.

### 6. Clustering using radius as features

Now, we use radius as the features and perform clustering. We will use [K-Means clustering](https://en.wikipedia.org/wiki/K-means_clustering). However, there is one **drawback** that the number of clusters should be known beforehand. `gmx_clusterByFeatures` implements several [cluster metrics](https://gmx-clusterbyfeatures.readthedocs.io/en/latest/commands/cluster.html#cmetric-prior) and also automatated way to decide number of clusters. We will use Elbow method to automatically determine the number of clusters.

Following command will perform the clustering of conformations on the basis of cavity-radius profile. Explanation of options are as follows:
* `-method kmeans`: Use K-Means clustering algorithm
* `-ncluster 15`: K-Means clustering will be performed for 1 upto 15 clusters times each time. Finally, based on `-ssrchange` option, final number of clusters will be automatically selected.
* `-cmetric ssr-sst`: Use Elbow method to decide final number of clusters.
* `-nfeature 23`: Take 23 features from the feature file. Here it is 23 slabs for which cavity radius is dumped in feature file `cavity_features.xvg`.
* `-sort features`: Sort the output clustered trajectory based on the distance in feature space from its central structure.
* `-ssrchange 1.5`: Threshold (percentage) of change in SSR/SST ratio in Elbow method to automatically decide the number of clusters.
* `-g cluster.log`: Log file containing clustering information and cluster-mterics.

**index group order**

1. **First index** group - Output group of atoms in the central structures

2. **Second index** group - Group of atoms to calculate RMSD between central conformations of clusters as RMSD matrix, which is dumped in the **log file** with `-g` option. Here, it is Ligand without hydrogen atoms.
    
3. **Third index** group - Used for Superimposition by least-square fitting. ONLY used in separate clustered trajectories to superimpose conformations on the central structure. 


**Content of the output `-g clustering-radius.log` file**

It contains the command summary, and for each input cluster-numbers, number of frames in each clusters. At the end it dumps the **Cluster Metrics Summary**, which is important for deciding final number of clusters.

In [3]:
%%bash

mkdir clustering-radius-output

echo 0 10 11 | gmx_clusterByFeatures cluster -f traj_superimposed.xtc -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx \
                                            -feat cavity_features.xvg -method kmeans -nfeature 23 -cmetric ssr-sst -ssrchange 1.5 -ncluster 15 \
                                            -cpdb clustering-radius-output/central.pdb -g clustering-radius.log -clid cluster-id-radius.xvg

mkdir: cannot create directory ‘clustering-radius-output’: File exists
 :-) GROMACS - gmx_clusterByFeatures cluster, 2025.0-dev-20250210-6949615-local (-:

Executable:   gmx_clusterByFeatures cluster
Data prefix:  /project/external/gmx_installed
Working dir:  /home/raj/workspace/gmx_clusterByFeatrues/tutorials/protein-pca
Command line:
  'gmx_clusterByFeatures cluster' -f traj_superimposed.xtc -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx -feat cavity_features.xvg -method kmeans -nfeature 23 -cmetric ssr-sst -ssrchange 1.5 -ncluster 15 -cpdb clustering-radius-output/central.pdb -g clustering-radius.log -clid cluster-id-radius.xvg


         :-)  gmx_clusterByFeatures cluster (-:

             Author: Rajendra Kumar

       Copyright (C) 2018-2019  Rajendra Kumar


gmx_clusterByFeatures is a free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at y

  Cluster Log output   

Command:
'gmx_clusterByFeatures cluster' -f traj_superimposed.xtc -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx -feat cavity_features.xvg -method kmeans -nfeature 23 -cmetric ssr-sst -ssrchange 1.5 -ncluster 15 -cpdb clustering-radius-output/central.pdb -g clustering-radius.log -clid cluster-id-radius.xvg

Choose a group for the output:
Selected 0: 'System'

Choose a group for clustering/RMSD calculation:
Selected 10: 'r_5-535_&_MainChain'

Choose a group for fitting or superposition:
Selected 11: 'r_5-535_&_MainChain_&_!r_72-85_&_!r_257-268_&_!r_338-352_&_!r_435-444'


 Input Trajectory dt = 100 ps



###########################################
########## NUMBER OF CLUSTERS : 1 ########
###########################################

Cluster-ID	TotalFrames
1		22501



###########################################
########## NUMBER OF CLUSTERS : 2 ########
###########################################

Cluster-ID	TotalFrames
1		11602
2		10899



#################

Reading frame       9 time 407500.000    
Back Off! I just backed up cluster-id-radius.xvg to ./#cluster-id-radius.xvg.2#
Reading frame       9 time 407500.000    
GROMACS reminds you: "Christianity may be OK between consenting adults in private but should not be taught to young children." (Francis Crick)





Cluster-ID	Central Frame	Total Frames 
1		19488		4230
2		9807		3259
3		1873		3048
4		14884		2659
5		20134		2538
6		13371		2271
7		22103		1482
8		2417		1405
9		21652		811
10		4075		798



Extracting coordinates of the central structure...


Calculating RMSD between central structures...


 Central structurs - RMSD matrix 
    c1     c2     c3     c4     c5     c6     c7     c8     c9    c10 
 0.000  0.162  0.219  0.173  0.115  0.213  0.169  0.242  0.167  0.233 
 0.162  0.000  0.172  0.174  0.160  0.148  0.159  0.184  0.161  0.183 
 0.219  0.172  0.000  0.231  0.221  0.188  0.190  0.142  0.204  0.154 
 0.173  0.174  0.231  0.000  0.166  0.220  0.162  0.241  0.169  0.230 
 0.115  0.160  0.221  0.166  0.000  0.213  0.164  0.247  0.159  0.242 
 0.213  0.148  0.188  0.220  0.213  0.000  0.214  0.175  0.208  0.174 
 0.169  0.159  0.190  0.162  0.164  0.214  0.000  0.213  0.127  0.211 
 0.242  0.184  0.142  0.241  0.247  0.175  0.213  0.000  0.229  0.140 
 0.167  0.161  0.204  0.169  0.159  

### 7. Analysis and visualizations

Now, we will perform few analysis and visualizations.

1. Calculating and plotting average cavity radius cluster-wise
2. Visualizing clustered cavity shapes

#### 1. Calculating and plotting average cavity radius cluster-wise

`gmx_clusterByFeatures holeclustersplot` can be used to plot the average radius of each clusters. The options to the sub-commands are as follows:
* `-xmin` - minimum value of cavity/channel axis position to consider
* `-xmax` - maximum value of cavity/channel axis position to consider
* `-endrad` - ignore if radius is greater than this value (mostly at the opening end)
* `-gap` - slab size along the cavity/channel
* `-ax` - X, Y or Z axis along which cavity/channel axis is aligned
* `-dl` - Do not consider last two smallest clusters for calculation and plotting.
* `-csv radius-clusterwise.csv` - Dump the average and standard-deviation of each clusters' radius to a csv file
* `-o radius-cluster.png` - Plot the average radius values from each cluster

In [4]:
%%bash

gmx_clusterByFeatures holeclustersplot -i radius.dat -clid cluster-id-radius.xvg -o radius-cluster.png -csv radius-clusterwise.csv \
                                       -xmin 5.5 -xmax 17 -endrad 6 -gap 0.5 -ax Y -dl 2 -wd 8


Reading file : radius.dat
Reading frame 0 at time        0.000
Maximum axis range is: (5.500 to 17.000)
Reading frame 20000 at time  2000000.000
Finishid reading.... Total number of frame read =  22501
After removing axis points with any missing data:
------------------------------------------------------------
#        Axis        Mean radius     Std. deviation
------------------------------------------------------------
       5.500            1.629            0.384
       6.000            1.750            0.451
       6.500            1.907            0.534
       7.000            2.081            0.594
       7.500            2.146            0.631
       8.000            2.142            0.637
       8.500            2.127            0.610
       9.000            2.111            0.581
       9.500            2.085            0.569
      10.000            2.036            0.571
      10.500            1.984            0.567
      11.000            1.948            0.559
      11.

<img src="radius-cluster.png" width="900"/>  

#### 2. Visualizing clustered cavity shapes

To visualize the cavity shape of each cluster, we will use central structure pdb file and calculate the radius with same options as we did it originally with whole trajectory.
This step also generate a pdb file with `-pdb` option containg cavity shape. 

Subsequently, we will use `sph_process` and `sos_triangle` tools (provided with HOLE package) to generate a file that can be used to draw the cavity in the **VMD**.

Note: Remove `%%capture --no-stdout` and `%%capture --no-stderr` to populate all the output and errors generated from these commands. 

In [5]:
%%capture --no-stderr
%%capture --no-stdout
%%bash

for i in 1 2 3 4 5 6 7 8
do
input=central_c${i}

echo 11 1 | gmx_clusterByFeatures hole -f clustering-radius-output/${input}.pdb -s inputs/ref_cavity.pdb -n inputs/onlyProtein.ndx -rad bondi \
                                       -catmid 1903 -cvect 0 1 0 -endrad 12 -sample 0.5 -pdb ${input}_sph -o dummy_radius.dat
mv ${input}_sph.pdb ${input}.sph

rm ${input}.sos ${input}.vmd_plot dummy_radius.dat
sph_process -sos -dotden 15 -color ${input}.sph ${input}.sos
sos_triangle -s < ${input}.sos > cavity-snapshots/${input}.vmd_plot
rm ${input}.sph ${input}.sos
done

**Visualizing cavity shape using VMD**

Above commands generate `cluster_<cluster_id>.vmd_plot` files that can be loaded in VMD to visualize the cavity.

1. Go to the cavity-snapshots library
2. At first load the central strucure by using follwoing command: `vmd ../clustering-radius-output/central_c1.pdb -e snap.vmd`. This command will load central structure of first crystal structure and sets several representations.
3. In VMD Main Window, click on `Analysis -> Extensions -> Tk Console`. It will open the Tk console. Write `source central_c1.vmd_plot` and it will plot the cavity.

<img src="https://raw.githubusercontent.com/rjdkmr/gmx_clusterByFeatures/master/docs/images/cavity-cluster-shapes.png" width="900"/>